# Introduction 
 A second attempt at Spaced Repeititon

In [ ]:
# Run the command 
# pip install --force-reinstall -r '..\paideia\requirements.txt'

In [3]:
pip install openai

     ---------------------------------------- 0.0/65.6 kB ? eta -:--:--
     ---------------------------------------- 65.6/65.6 kB 3.7 MB/s eta 0:00:00
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached typing_extensions-4.9.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
   ---------------------------------------- 0.0/222.3 kB ? eta -:--:--
   ---------------------------------------- 222.3/222.3 kB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/85.5 kB ? eta -:--:--
   ---------------------------------------- 85.5/85.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/75.9 kB ? eta -:--:--
   ---------------------------------------- 75.9/75.9 kB 4.1 MB/s eta 0:00:00
Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
   --------------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install BeautifulSoup

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "c:\Users\jbay\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "c:\Users\jbay\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\jbay\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\jbay\AppData\Local\Temp\pip-build-env-b3ra7umr\over

In [1]:
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
import regex as re
import time
import numpy as np
import random
import pyperclip as py
# import PyPDF2
import openai 
import regex as re
import random 
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from requests.exceptions import MissingSchema
import tiktoken # https://github.com/openai/tiktoken

from dotenv import load_dotenv
load_dotenv()
openai.api_type  = os.getenv('OPENAI_TYPE')
openai.api_base  = os.getenv('OPENAI_BASE')
openai.api_version = os.getenv('OPENAI_VERSION')
openai.api_key = os.getenv('OPENAI_KEY')

%autosave 60 

Autosaving every 60 seconds


In [2]:
def vector_similarity(vec1,vec2):
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(vec1), np.array(vec2))

def calculate_similarity(vector):
    #print(type(vector),type(prompt_embedding))
    similarity = vector_similarity(vector, prompt_embedding)
    return similarity

def embed_prompt_lookup(question: str) -> str:
    # Your code remains unchanged until this line
    prompt_embedding = get_embedding(question)

    # Filter out rows with None values in the 'embedding' column
    valid_new_df = new_df[new_df['embedding'].notna()]

    # Get prompt similarity with embeddings
    new_df['prompt_similarity'] = np.nan  # Initialize a new column with NaN values

    for index, row in valid_new_df.iterrows():
        vector = list(row['embedding'])
        similarity = vector_similarity(vector, prompt_embedding)
        new_df.at[index, 'prompt_similarity'] = similarity
        
    # Rest of the code remains unchanged
    # get most similar summary
    top_3_similar = df.nlargest(3, 'prompt_similarity')
    summaries = top_3_similar['line'].tolist()

    prompt = f"""Only answer the question below if you have 100% certainty of the facts, use the context below to answer.
            The answer should be written as an email for a colleague in my organization.
            The answer must include citing, in a footnotes format. Here's an example: '[n]'
            Here is some context:
            {summaries[0]}
            {summaries[1]}
            {summaries[2]}
            Q: {question} \n\n
            A:"""

    # Use OpenAI to get the most similar answer
    response = openai.Completion.create(
        engine = 'text-davinci-003', # Use the text-davinci-003 engine
        prompt=prompt, # Use the prompt created above
        temperature=0, # Set the temperature to 0
        max_tokens=500, # Set the maximum tokens to 500
        model="text-davinci-003" # Use the text-davinci-003 model
    )
    print(response["choices"][0]["text"].strip(" \n")) # Print the most similar answer```

def get_embedding(text):
    tries = 0
    if isinstance(text, str):
        pass
    else:
        return None
    try:
        result = openai.Embedding.create(
            engine='text-embedding-ada-002',
            model='text-embedding-ada-002',
            input=text
        )
        return result["data"][0]["embedding"]
    except Exception as e:
        #print(f"Error: {e}")
        time.sleep(7)
        if tries < 3:
            get_embedding(text)
            tries += 1
            #print(tries)
        else:
            return None

def weird_func(string, encoding_name):
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def read_pdf_to_string(file_path):
    """Reads a PDF file and returns the text as a string.

    Args:
        file_path (str): The path to the PDF file.

    Returns:
        str: The text from the PDF file.
    """
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)  # Create a PDF reader object
        num_pages = len(pdf_reader.pages)  # Get the number of pages in the PDF
        text = ""

        for page_number in range(num_pages):  # Iterate through each page
            page = pdf_reader.pages[page_number]  # Get the page object
            text += page.extract_text()  # Extract the text from the page

    return text  # Return the text from the PDF

def cleaner(text):
    if isinstance(text, str):
        list = []

        text = text.replace('[', '')
        text = text.replace(']', '')

        for j in text.split(','):
            j = j.strip()
            list.append(float(j))
        return list
    else:
        return text

def my_reader(path):
    embeddings = []
    df = pd.read_csv(path)
    for i in range(len(df)):
        x = df.at[i, 'embedding']
        y = cleaner(x)
        embeddings.append(y)
        
    df['embedding'] = embeddings
    return df

def split_response(response, max_chars_per_line=500):
    words = response.split(' ')
    lines = []
    current_line = words[0]

    for word in words[1:]:
        if len(current_line + " " + word) < max_chars_per_line:
            current_line += " " + word
        else:
            lines.append(current_line.strip())
            current_line = word

    lines.append(current_line.strip())
    return lines

def get_links_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extract all the links (a tags) from the page
        links = soup.find_all("a")
        return links
    except MissingSchema:
        return []

def process_links(links, visited_links):
    new_links = []
    for link in links:
        href = link.get("href")
        if href:
            if href.startswith('/hc/'):
                href = 'https://support.steelseries.com' + href
            if ('steelseries' in href) and (href not in visited_links):
                visited_links.add(href)
                new_links.append(href)
    return new_links, visited_links

def fetch_all_links(url):
    visited_links = set()
    to_visit = [url]

    with tqdm(total=10000, desc="Links Processed") as pbar:
        while to_visit:
            try:
                if len(visited_links) > 10000:
                    break

                current_url = to_visit.pop()
                links = get_links_from_url(current_url)
                new_links, visited_links = process_links(links, visited_links)
                to_visit.extend(new_links)
                pbar.set_description(f"Visited: {len(visited_links)}, To visit: {len(to_visit)}")
                pbar.update(1)
            except Exception as e:
                print(e)
                return visited_links
    return visited_links

def get_text_from_url(url):
    try:
        
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Remove script and style tags
        for script in soup(["script", "style"]):
            script.decompose()

        # Extract the text directly using .get_text() method
        raw_text = soup.get_text(separator='\n')

        # Remove consecutive line breaks
        lines = raw_text.split('\n')
        filtered_lines = [line for line in lines if line.strip() != '']
        text = '\n'.join(filtered_lines)
        return text
    except:
        print('error')
        return ''

def get_redirect(url):
    response = requests.head(url, allow_redirects=True)

    if response.status_code == 200:
        #print('The link is valid and returns a 200 status code.')
        final_url = response.url
        #print(f'The final URL is {final_url}.')
    else:
        #print('The link is invalid or does not return a 200 status code.')
        final_url = url
    return final_url

def embed_prompt_lookup(question: str, key: str, df: pd.DataFrame) -> str:
    #key = 'article'
    prompt_embedding = get_embedding(question)

    df['prompt_similarity'] = np.nan  

    if len(key) > 3:
        definition              = df[df.Name_ == sample_name].text.values[0]
        definition_embedding    = df.loc[df.Name_ == sample_name, 'embedding'].values[0]
        prompt_similarity       = vector_similarity(definition_embedding,
                                                    prompt_embedding)
        most_similar_summary    = definition    
    else:
        # Initialize a new column with NaN values

        for index, row in df.iterrows():
            if pd.notna(row['embedding']).all():
                df.at[index, 'prompt_similarity'] = vector_similarity(list(row['embedding']), prompt_embedding)

        # get most similar summary
        most_similar_summary = df.nlargest(1, 'prompt_similarity')['text'].values[0]
        prompt_similarity = df.nlargest(1, 'prompt_similarity')['prompt_similarity'].values[0]

    # Rest of your code...

    prompt = f"""
            You will act as an experienced Teacher, helping a student prepare for an exam.
            I will provide you with a statement, and a the definition. 
            You will start your reply with a score from 0-100, in how close you evaluate it is to the definition.
            If you based on the reading of the definition and statement, find a difference, then please reply with an argument for the discrepancy.
            The format is:
            eval_number/100 \n [your reply]
            Here is some context:
            {most_similar_summary}
            Q: {question} \n\n
            A:"""
    # Use OpenAI to get the most similar answer
    response = openai.Completion.create(
        engine = 'text-davinci-003', # Use the text-davinci-003 engine
        prompt=prompt, # Use the prompt created above
        temperature=0, # Set the temperature to 0
        max_tokens=500, # Set the maximum tokens to 500
        model="text-davinci-003" # Use the text-davinci-003 model
        )
    response = response["choices"][0]["text"].strip(" \n")
    print(prompt_similarity)
    return response, most_similar_summary

def set_embeddings(df, df_path):
    if 'embeddings' in df.columns or 'embedding' in df.columns:
        pass
    else:
        embedding = []
        # Wrap the DataFrame's 'summary' column with tqdm to show progress bar
        for text in tqdm(df['text'], desc="Getting embeddings"):
            new_embedding = get_embedding(text)
            embedding.append(new_embedding)

        # We need to do it again, because the recursion doesn't work fully
        for i in tqdm(range(len(embedding))):
            if embedding[i] == None:
                summary = df.loc[i, 'text']
                embedding[i] = get_embedding(summary)

        df['embedding'] = embedding

        df.to_csv(df_path, index = False)
    return df

In [3]:
def print_performance(df, index):
    score = df.at[index, 'last_score']
    ps = df.at[index, 'last_ps']
    df.to_csv('moves.csv', index=False)
    if score > 90:
        print(f"Excellent job! You're mastering this material. DaVinci rated you {score}, Your PS value: ", ps)
    elif score > 70:
        print(f"Good work! Keep practicing to improve even more. DaVinci rated you {score}, Your PS value: ", ps)
    elif score > 50:
        print(f"You're making progress, but there's still room for improvement. DaVinci rated you {score}, Your PS value: ", ps)
    else:
        print(f"Don't be discouraged. Keep studying and you'll get there! DaVinci rated you {score}, Your PS value: ", ps)

def store_date(key,
               text,
               category,
               embedding,
               last_score,
               last_runtime,
               last_ps,
               prompt_similarity):
    path = 'storing_df.csv'
    
    storing_df = pd.read_csv(path)
    storing_df.loc[len(storing_df)] = [key,
                                        text,
                                        category,
                                        embedding,
                                        last_score,
                                        last_runtime,
                                        last_ps,
                                        prompt_similarity]
    storing_df.to_csv(path, index=False)

def find_rating(x: str):
    if '100/100' in x:
        return 100
    elif x.startswith('0/100'):
        return 0
    else:
        return re.findall('\d{2}', x)[0]

def key_embed_prompt_lookup(question: str, key: str, df: pd.DataFrame) -> str:
    prompt_embedding = get_embedding(question)

    df['prompt_similarity'] = np.nan  

    if len(key) > 3:
        
        definition              = df[df.key == key].text.values[0]
        definition_embedding    = df.loc[df.key == key, 'embedding'].values[0]
        category                = df.loc[df.key == key, 'Category'].values[0]
        prompt_similarity       = vector_similarity(definition_embedding,
                                                    prompt_embedding)
        most_similar_summary    = definition    
    else:
        # Initialize a new column with NaN values
        for index, row in df.iterrows():
            if pd.notna(row['embedding']).all():
                df.at[index, 'prompt_similarity'] = vector_similarity(list(row['embedding']), prompt_embedding)

        # get most similar summary
        most_similar_summary = df.nlargest(1, 'prompt_similarity')['text'].values[0]
        prompt_similarity = df.nlargest(1, 'prompt_similarity')['prompt_similarity'].values[0]
        category           = df.nlargest(1, 'prompt_similarity')['Category'].values[0]    # Rest of your code...
    
    print(most_similar_summary)        
    #messages = [{"role":"system","content":"You will act as an experienced Teacher, assisting a student in exam preparation. I will provide you with a statement, and a corresponding definition. Your role is to assess the degree of similarity between the statement and the definition on a precise scale from 0-100, accuracy is the key, so please be very precise in your response. In case you find any discrepancy between the statement and the definition, please articulate your observation as a coherent argument"},
    messages = [{"role":"system","content":"""
                 You will act as an experienced Active Recall Expert , assisting me in evaluating my statements compared to the definitions, or as in assisting a student in exam preparation.
                 I will provide you with a statement, and a corresponding definition.
                 Your role is to assess the degree of similarity between the statement and the definition on a precise scale from 0-100, accuracy is the key, so please be very precise in your response.
                 In case you find any discrepancy between the statement and the definition, please articulate your observation as a coherent argument. 
                 If I reply I have no clue, please reply with a few hints to the answer, give me the score 0/100
                 """},
                {"role":"user","content":f"I am studying Retrieval-Augmented Generation and Im currently researching the below question/term Why use Retrival-Augmented Generation?, which is defined as: \n {'One of the core use cases of Large Language Models (LLMs) is question-answering over your own data. To do this, we pair the LLM with a “retrieval” model that can perform information retrieval over a knowledge corpus, and perform response synthesis over the retrieved texts using the LLM. This overall framework is called Retrieval-Augmented Generation.'} \n I define it as: \n Retrival-Augmented Generation is an extension of the typical embedding-retrival. However, by utilising LLM Retrival and embedding, we can get better performance, as the corpus of text provided for generation is greater, and can provide both the specific context and the global context \n "},
                {"role":"assistant","content":"70/100. \n\nWhile your statement correctly highlights the use of LLMs in Retrieval-Augmented Generation and the benefits of utilizing a larger corpus of text for generation, it does not fully capture the core concept of the framework. Retrieval-Augmented Generation is not just an extension of embedding-retrieval, but rather a specific framework that involves pairing an LLM with a retrieval model to perform question-answering over a knowledge corpus. Additionally, your statement does not mention the synthesis of responses over the retrieved texts using the LLM, which is a key aspect of the framework."},
                {"role":"user","content":f"I am studying Organisational Movements and Im currently need your help understanding the concept Pytlic, defined as: Making receivers of a presentation, guess what im thinking instead of providing or instructing them. Currently my understanding is: I dont know anything about it, can you give a hint"},
                {"role":"assistant","content":" 0/100 \n\n As you do not comprehend the definition at hand, your score is 0. \n Pytlick is the organisational move, when a presenter is requesting the audience to guess what they are thinking, instead of providing the answer them selves"},
                {"role":"user","content":f"I am studying {category} and Im currently researching the below question/term {key}, which is defined as: \n {most_similar_summary} \n I define it as: \n {question} \n "},
                ]   
         
    response = openai.ChatCompletion.create(
        deployment_id = 'gpt-35-turbo',
        engine = 'gpt-35-turbo',
        messages = messages,
        temperature = 0.2,
        max_tokens = 400,
        top_p = 0.95,
        frequency_penalty = 0,
        presence_penalty = 0,
        stop = None
    )
    response = response["choices"][0]['message']['content']
    print(prompt_similarity)
    
    ## Storing data for later retrieval
    
    store_date(key = key,
               text = question,
               category = df.loc[df.key == key, 'Category'].values[0],
               embedding=prompt_embedding,
               last_score=find_rating(response),
               last_runtime=int(datetime.utcnow().timestamp()),
               prompt_similarity =prompt_similarity,
               last_ps=prompt_similarity)
    
    
    return response, most_similar_summary,prompt_similarity

response = ''

def print_response(response = response):
    lines = response.splitlines()
    for line in lines:
        if len(line) > 100:
            sentences = line.split('. ')
            for sentence in sentences:
                sentence = sentence + '.'
                print(sentence) 
        else:
            print(line)

answer = 'I dont know'

key = 'Pytlick'

print(key)

Pytlick


In [6]:
df = my_reader('moves.csv')
df

,key,text,Category,embedding,last_score,last_runtime,last_ps,prompt_similarity
0,Jeppesen,"Asking about the process, saying how can we h...",Organisational Movements,"[-0.008856984786689281, 0.014992808923125267, ...",90,1683728976,0.885209,NaN
1,Why use Retrival-Augmented Generation?,One of the core use cases of Large Language Mo...,Retrieval-Augmented Generation,"[-0.026878967881202698, 0.027037078514695168, ...",70,1683790635,0.886467,NaN
2,How normal semantic synthesis is performed?,Most users building LLM-powered QA systems tod...,Retrieval-Augmented Generation,"[0.003434924641624093, 0.025081591680645943, -...",50,1683867733,0.823933,NaN
3,What is the big pitfall with embedding-retrival?,Text chunks lack global context. Oftentimes th...,Retrieval-Augmented Generation,"[0.008670363575220108, 0.007526219356805086, 0...",90,1683868201,0.859046,NaN
4,What are the three big tuning issues with embe...,Careful tuning of top-k / similarity score thr...,Retrieval-Augmented Generation,"[0.012233213521540165, -0.005706304218620062, ...",10,1683868329,0.728263,NaN
...,...,...,...,...,...,...,...,...
225,What is the basic concept of ChatGPT and why d...,So what would happen if we applied ChatGPT to ...,Generative Pre-trained Transformers,"[-0.015013879165053368, 0.0011765881208702922,...",0,0,0.000000,NaN
226,How does ChatGPT generate coherent human langu...,"As we’ve seen, the actual neural net in ChatGP...",Generative Pre-trained Transformers,"[-0.014149912633001804, 0.012706183828413486, ...",0,0,0.000000,NaN
227,How is ChatGPT able to generate human-like tex...,The specific engineering of ChatGPT has made i...,Generative Pre-trained Transformers,"[-0.010274942964315414, 0.01824106089770794, 0...",0,0,0.000000,NaN
228,How does the training strategy of ChatGPT diff...,When it comes to training (AKA learning) the d...,Generative Pre-trained Transformers,"[-0.021468183025717735, 0.0143800163641572, 0....",0,0,0.000000,NaN


In [7]:
new_df = pd.read_csv('data.csv')
new_df


,header,text,length,tokens,summary,question,category,Author,last_score,last_run
0,Tool Moderate Exercise before cognetive:,that I think you will find particularly intere...,6501.0,1305.0,"The study titled ""Brief Aerobic Exercise Immed...",What did the study by Legrand et al. find abou...,Cold Exposure,Andrew Huberman,100,1685322425
1,Circadian Rhythm & Body Temperature:,Let's talk about the use of cold for health an...,2798.0,569.0,The use of cold for health and performance has...,What is the relationship between deliberate co...,90,1684714574,1,1685322434
2,"Tool: Quickly Decrease Core Body Temperature, ...",So that's the circadian rhythm in temperature....,3195.0,694.0,Andrew huberman explains how the thermal regul...,How would the body react to getting cold water...,Cold Exposure,Andrew Huberman,0,1685321333
3,"What I'll tell you, and we'll get into this in...",We're going to talk more about the specific pr...,3095.0,652.0,The main takeaways from this lecture are that ...,What are the main takeaways from Andrew Huberm...,Cold Exposure,Andrew Huberman,0,1684715097
4,Mental Effects of Cold Exposure::,Mental Effects of Cold Exposure: And I want to...,3282.0,696.0,Deliberate cold exposure can improve mental pe...,What are the benefits of deliberate cold expos...,Cold Exposure,Andrew Huberman,80,1684715321
5,Physical Effects of Cold Exposure:,but deliberate cold exposure has also been stu...,1253.0,252.0,Deliberate cold exposure has been studied in a...,What are the potential benefits of deliberate ...,Cold Exposure,Andrew Huberman,80,1684715790
6,How Cold Should the Temperature Be?:,but where I'd like to start is with mental per...,4934.0,1030.0,NaN,What is the key point that Andrew Huberman mak...,Cold Exposure,Andrew Huberman,0,0
7,Cold Showers vs. Cold Water Immersion:,"cold showers are as good, better or worse than...",3756.0,806.0,Cold water immersion up to the neck with hands...,What are the most effective forms of cold expo...,Cold Exposure,Andrew Huberman,0,1684715885
8,Protocols for Cold Exposure:,and performance using deliberate cold exposure...,12897.0,2773.0,Deliberate cold exposure can increase norepine...,What is the recommended approach for increasin...,Cold Exposure,Andrew Huberman,90,1684716011
9,Optimal Mindset(s) During Cold Exposure:,The next question that I always get is what sh...,4796.0,976.0,is the importance of gradually increasing the ...,What are the potential hazards of deliberate c...,Cold Exposure,Andrew Huberman,80,1684716089


In [17]:
for i in range(len(df)):
    if i == 0:
        pass
    else:
        x = df.loc[i]
        header = ""
        text = x['text']
        length = len(x['text'])
        tokens = int(len(x['text'])/4)
        summary = ""
        question = x['key']
        author = ""
        category = x['Category'] 
        last_score = x['last_score']
        last_run = x['last_runtime']
        
        new_df.loc[len(new_df)] = [header, text, length, tokens, summary, question, category, author, last_score, last_run]
        

In [93]:
def message_cleaner(messages:list = messages) -> list:
    tmp = []

    for j in messages:
        new_line = {'role': '',
                    'content': ''}
        for k in j:
            if k == 'content':
                x = re.sub(' {2,}', '', j[k])
                x = x.strip()
                new_line['content'] = x 
            else: 
                new_line['role'] = j[k]
                
        tmp.append(new_line)
                
    return tmp

def api_call(y = ''):
    x = random.randint(0, 100)
    
    return x , y

def marseille_api_call(problem = '', key = '', j = ''):
    #x = random.randint(0, 100)
    x = 0
    messages = [{"role":"system","content":"""
                I want you to act as a SQL Teacher.
                You will assist me with my questions, by giving me hints to the correct solution to my questions. 
                I will try three times each, and then you will give me the solution, i will start each message with my attempt number.
                Once i reach the solution you will and explain step by step what happens in the key.
                You should use your knowledge of SQL and database management to provide me with the best solutions.
                In the event, that i write a solution which solves the problem, but is not the best solution, please congratulate me, and then write the best practice. 
                
                Here is an example 
                User:
                Attempt: 1
                Problem: Finding the amount of orders completed within a period
                Key: 
                Select orders, timestamp, and product_sku
                From table;
                
                Assistant:
                Nice job, you got the correct values, however to get the count you need to use the COUNT function
                
                User:
                Attempt: 2
                Problem: Finding the amount of orders completed within a period
                Key: 
                Select count(orders) as order,
                product_sku as product,
                timestamp as timestamp
                from 
                table
                where timestamp between 07-05-2023 and 15-05-2023
                order by product;
                
                Assistant:
                Great job! You got it 
                """},
                {'role': 'user', 
                 'content': f'Attempt: {j}\nProblem: {problem}\n: Key: {key}'}
                ] 

    messages = message_cleaner(messages=messages)
    
    y = ''
    return x , y

def brute_sleep(release_int = 5):
    now = int(datetime.utcnow().timestamp())
    release = now + release_int
    print('im working')
    while now <= release:
        now = int(datetime.utcnow().timestamp())

def api_rec(x = '', y: int = 0, j: int = 0):
    if x == '':
        x = input()
    
    
    y , x  = api_call(y = x)
    print(j)
    if j >= 3:
        print( 'to many attempts')
    else: 
        if y > 70:
            print(y, x)
        else:
            x = input()
            j = j + 1 
            print(x, y, j)
            apo_rec(x, y, j)

def m_rec(x = '', y: int = 0, j: int = 0):
    if x == '':
        x = input()
    
    
    y , x  = marseille_api_call(problem = 'hej',
                                key = 'not my problem',
                                j = j)
    print(j)
    if j >= 3:
        print( 'to many attempts')
    else: 
        if y > 70:
            print(y, x)
        else:
            brute_sleep(3)
            x = input()
            print(x, y, j)
            m_rec(x, y, j)
            j = j + 1 
            
m_rec('', y = 1, j = 0)

0
im working
2 0 0
0
im working
 0 0


KeyboardInterrupt: Interrupted by user

In [4]:
new_df = pd.read_clipboard(header=None)

new_df

,0,1,2,3
0,Product,NaN,SKU Number,https://purview.microsoft.com/datacatalog/gove...
1,Product,NaN,Product Description,[Product Platform Document or Description -> h...
2,Product,NaN,Part Type,NaN
3,Product,NaN,Product Group Code,https://purview.microsoft.com/datacatalog/gove...
4,Product,NaN,Product Type Code,NaN
...,...,...,...,...
126,Product,Financial,Item Group (Global),NaN
127,Product,Financial,Item Sales Tax Group (Local),NaN
128,Product,Financial,Financial Dimensions (Global),NaN
129,Product,Financial,Comodity Code --> UN Commodity Code (Global),NaN


In [5]:
new_df.columns = ['Data Object', 'Data Object Child', 'Data Object Attribute', 'Attribute Description']

new_df

,Data Object,Data Object Child,Data Object Attribute,Attribute Description
0,Product,NaN,SKU Number,https://purview.microsoft.com/datacatalog/gove...
1,Product,NaN,Product Description,[Product Platform Document or Description -> h...
2,Product,NaN,Part Type,NaN
3,Product,NaN,Product Group Code,https://purview.microsoft.com/datacatalog/gove...
4,Product,NaN,Product Type Code,NaN
...,...,...,...,...
126,Product,Financial,Item Group (Global),NaN
127,Product,Financial,Item Sales Tax Group (Local),NaN
128,Product,Financial,Financial Dimensions (Global),NaN
129,Product,Financial,Comodity Code --> UN Commodity Code (Global),NaN


In [7]:
import utilities as my

In [8]:
new_df = my.rename_columns(new_df)

new_df

,data_object,data_object_child,data_object_attribute,attribute_description
0,Product,NaN,SKU Number,https://purview.microsoft.com/datacatalog/gove...
1,Product,NaN,Product Description,[Product Platform Document or Description -> h...
2,Product,NaN,Part Type,NaN
3,Product,NaN,Product Group Code,https://purview.microsoft.com/datacatalog/gove...
4,Product,NaN,Product Type Code,NaN
...,...,...,...,...
126,Product,Financial,Item Group (Global),NaN
127,Product,Financial,Item Sales Tax Group (Local),NaN
128,Product,Financial,Financial Dimensions (Global),NaN
129,Product,Financial,Comodity Code --> UN Commodity Code (Global),NaN
